#### Assigning taxonomy to nanopore reads

In [2]:
#loading packages
library(tidyverse)  

In [3]:
#loading reference file (linking refseq accession number and taxID
#reference file was obtaining parsing gff3 file provided by NCBI 
RefseqTaxID <- read.delim("data/RefseqTaxID.txt", h=T)

#loading TaxID full lineage file 
#Check out step 2-Parsing_Database_Refseq16S
TaxID_FullLineage <- read.delim("data/referencetable_taxonomy_RefseqNCBI_16S.txt", quote = "")

#loading metadata file including read counts after quality check
metadata <- read.delim("data/metadata_nreads.tsv", h=T)

In [4]:
#### REPO with kma output
fragfiles <- list.files(pattern = ".frag$")

In [82]:
dfall <- NULL

for(i in 1:length(fragfiles)){
    tab <- read.delim(fragfiles[i], h=F)
    freqtab <- table(gsub(" .*", "", tab$V6))
    df <- data.frame(Refseq = names(freqtab), Counts = as.numeric(freqtab))
    dfTaxID <- merge(RefseqTaxID, df, by = "Refseq")
    freqtable_tax <- merge(dfTaxID, TaxID_FullLineage, by.x = "TaxID", by.y = "id")
    freqtable_tax$ID_Sample <- gsub("-.*", "", fragfiles[i])
    freqtable_tax_metadata <- merge(freqtable_tax, metadata, by = "ID_Sample")
    freqtable_tax_metadata$RelativeAbundance <- ((freqtable_tax_metadata$Counts)/sum(freqtable_tax_metadata$Counts))*100
    freqtable_tax_metadata$FreqAlignment <- (sum(freqtable_tax_metadata$Counts)/freqtable_tax_metadata$NumberReads)*100
    dfall <- rbind(dfall, freqtable_tax_metadata)
}

In [73]:
write.table(dfall, "ReadCounts_TaxAnnotation.txt", quote = F, sep = "\t", row.names = F)